# The Numeric Widgets

#### `Int` Widgets

+ IntText
+ BoundedIntText
+ IntProgress
+ IntSlider
+ IntRangeSlider

#### `Float` Widgets

+ FloatText
+ BoundedFloatText
+ FloatProgress
+ FloatSlider
+ FloatRangeSlider

----------------------

First of all, we have to import the Widgets library. Sometimes we need some Haskell language extensions. We need `OverloadedStrings` because some widget methods receive a `Text` as an argument, instead of a `[Char]`.

In [ ]:
{-# LANGUAGE OverloadedStrings #-}
import IHaskell.Display.Widgets

#### `IntText` and `FloatText`
We are going to see how the `Text` family works. They create a "Stepper", which lets us click to increment a certain amount. We can also input the number as if it were text.

In [ ]:
intt <- mkIntText
floatt <- mkFloatText
intt
floatt

For any numeric widget, we can use `getField w IntValue` and `getField w FloatValue` to obtain the value stored in the widget. Try changing the numbers in the widgets and then executing the cell bellow!

In [ ]:
getField intt IntValue
getField floatt FloatValue

We can also use these results for any computation. For example, let us add them together:

In [ ]:
x <- getField intt IntValue
y <- getField floatt FloatValue

(fromIntegral x) + y

But incrementing and decrementing one by one is a bit boring... We can change the number incremented each step if we set the `StepInt` and `StepFloat` fields.

In [ ]:
setField intt StepInt 5
setField floatt StepFloat 0.5

Try clicking on the buttons of the text field now.

By the way, does this mean that we can change other fields with `setField`? Yes! For example, let us change the `IntValue`/`FloatValue`

In [ ]:
setField intt IntValue 42
setField floatt FloatValue 3.14

One special field is the `ChangeHandler`. This field is an IO function that is executed everytime the value is changed. We are going to "sync" the two widgets using two `ChangeHandler` functions. Each time the value changes in one widget, it is changed on the other widget too.

In [ ]:
setField intt ChangeHandler (getField intt IntValue >>= setField floatt FloatValue . fromIntegral)
setField floatt ChangeHandler (getField floatt FloatValue >>= setField intt IntValue . round)

-- Let's display the widgets again so we don't have to scroll up and down
intt
floatt

#### `BoundedIntText` and `BoundedFloatText`

So, what's the difference between the Bounded family and the other two? `Bounded` widgets have the `Max` and `Min` attributes, which let you set an upper and lower bound. Let's try it

In [ ]:
-- We create the two widgets
bit <- mkBoundedIntText
bft <- mkBoundedFloatText

setField bit MaxInt 5
setField bit MinInt (-5)

setField bft MaxFloat 20
setField bft MinFloat 10
setField bft StepFloat 0.5
setField bft ChangeHandler (getField bft FloatValue >>= print)
setField bft FloatValue 15

bit
bft

If you try clicking on the buttons or editing the text, you can't but anything that is not between the `Min` and `Max`! That's pretty neat, but, if you want to do something with bounds, maybe it's better to use an slider

#### `IntSlider`, `FloatSlider`, `FloatLogSlider`, `IntRangeSlider` and `FloatRangeSlider`

All these widgets are sliders (duh!). `IntSlider`, `FloatSlider` and `FloatLogSlider` represent a single value, whereas `IntRangeSlider` and `FloatRangeSlider` represent a pair (range) of values. `FloatLogSlider` uses a logarithmic scale, which means that every step will multiply (instead of increment) the value!

In [ ]:
ins <- mkIntSlider
irs <- mkIntRangeSlider
fns <- mkFloatSlider
fls <- mkFloatLogSlider
frs <- mkFloatRangeSlider
setField fns StepFloat 0.25
-- We can set the base of the logslider
setField fls BaseFloat 2
setField fls StepFloat 1

In [ ]:
ins
irs
fns
fls
frs

They work the same as the previous widgets plus, they are a lot more confortable if you have bounds. The only difference is with the pair ones, we have to get the `IntPairValue` and `FloatPairValue` fields

In [ ]:
getField irs IntPairValue
getField frs FloatPairValue

In [ ]:
-- We can also set the field!
setField irs IntPairValue (32,42)

Let's create a small program using widgets that gives us the greatest common divisor of two numbers! It displays the result on a `IntText` widget

In [ ]:
gcd a 0 = a
gcd a b = gcd b $ a `mod` b

setField irs ChangeHandler (getField irs IntPairValue >>= setField intt IntValue . uncurry gcd)

irs
intt

#### `IntProgress` and `FloatProgress`

Finally, we have these two widgets, that we can use as progress bars.

In [ ]:
fnp <- mkFloatProgress
fnp

In [ ]:
setField fnp FloatValue 42.5
getField fnp FloatValue

We can also display them Vertically.

*(Did you know you can also display the Sliders vertically?)*

In [ ]:
setField fnp Orientation VerticalOrientation

Now we are going to create a REAL progress bar! For example, let's create a thread that does some incredible complex calculations (sleeping) and then reports the progress to the widget

In [ ]:
import Control.Concurrent
import System.IO.Unsafe

inp <- mkIntProgress
inp

In [ ]:
f :: Integer -> IO ()
f x = do
    threadDelay (5*10^4)
    setField inp IntValue x
   
thid <- forkIO $ mapM_ f [0..100]